In [45]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import preprocessing
from datetime import datetime
from sklearn.metrics import r2_score

I import all the necessary libraries. There are many machine learning libraries for neural network. I picked sklearn becuase I'm most familiar and comfortable with this.

In [46]:
df = pd.read_csv('Climate_dataset_TS.csv')

I use pandas to read Climate_dataset_TS.csv and store them as df

In [47]:
le = preprocessing.LabelEncoder()
df['date'] = le.fit_transform(df['date'])

I categorized the date to number for calculating.
The dates in df are all organized low to high. The earliest date in df is 2017-01-01, so the encoder will encode it as 0. There 114 rows of data. The last date is 2017-04-24, so the encoder will encode it as 113.

In [48]:
X = df[['date', 'wind_speed', 'meanpressure']]
y = df[['meantemp', 'humidity']]

I separate the data into X and y by column. olumns in X are 'date', 'wind_speed', and 'meanpressure' and columns in y are 'meantemp' and 'humidity'.


In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

I split the data into testing and training. 80% test and 20% train.

In [50]:
# nnTempHumid = MLPRegressor(activation='relu', solver='lbfgs', max_iter=15000).fit(X_train, y_train)
nnSpeedPressure = MLPRegressor(activation='relu', solver='lbfgs', max_iter=15000).fit(df[['date']], df[['wind_speed', 'meanpressure']])
nnTempHumid = MLPRegressor(activation='identity', solver='lbfgs', max_iter=15000).fit(X_train, y_train)
# nnSpeedPressure = MLPRegressor(activation='identity', solver='lbfgs', max_iter=15000).fit(df[['date']], df[['wind_speed', 'meanpressure']])

I imported sklearn.MLRegressor to train my data </br>
The parameters are:
- activation='relu' for wind speed and mean pressure and 'identity' for windspeed and mean pressure - because all I think this fit the most. logistic is for classification, tanh might not fit becuase it is more linear. I picked them based on their r2 score
- solver='lbfgs' - becuase I want the training process to stop when loss value has reached the threshold of 1e-4, instead of training for too much or too little, and free up computing time.
- max_iter=15000 - if the interation reached 15000 then it will not continue instead of waiting for the loss to reach the threshold.
</br>
I train for predicting mean temperature and humidity, with 'date', 'wind_speed', and 'meanpressure' as input
</br>
I train for predicting mean ''wind_speed' and 'meanpressure', with 'date' as input

In [51]:
predictTempHumid = nnTempHumid.predict(X_test)
scoreTempHumid = nnTempHumid.score(X_test, y_test)
predictSpeedPressure = nnSpeedPressure.predict(X_test[['date']])
scoreSpeedPressure = nnSpeedPressure.score(X_test[['date']], X_test[['wind_speed', 'meanpressure']])

I calculate the score and predict the value to compare with the original values.

In [52]:
print('Score for predicting Temperature and Humidity', scoreTempHumid)
for a, b in zip(predictTempHumid, y_test.values.tolist()):
    print(a, b)
print('Score for predicting Wind Speed and Mean Pressure', scoreSpeedPressure)
for a, b in zip(predictSpeedPressure, X_test[['wind_speed', 'meanpressure']].values.tolist()):
    print(a, b)

Score for predicting Temperature and Humidity -0.78571040552771
[26.2929604  40.37745688] [29.75, 33.75]
[26.44575858 40.73831372] [30.5, 29.75]
[26.57500732 37.5990831 ] [30.933333333333334, 31.866666666666667]
[26.73088516 37.09725672] [29.23076923076923, 46.0]
[26.88780407 36.90976949] [31.22222222222222, 26.0]
[27.03692443 39.65372141] [27.0, 29.875]
[27.17157895 37.84718052] [25.625, 29.375]
[27.31141715 35.28220574] [27.125, 21.125]
[27.45914304 32.93212575] [27.857142857142858, 19.428571428571427]
[27.6212045  33.82474397] [29.25, 17.75]
[27.78070525 35.51993479] [29.25, 26.0]
[27.9341009  36.68111292] [29.666666666666668, 29.11111111111111]
[28.0952825  37.48548768] [30.5, 37.625]
[28.24431125 35.35541103] [31.22222222222222, 30.444444444444443]
[28.39698451 36.27260043] [31.0, 34.25]
[28.53774918 34.61716503] [32.55555555555556, 38.44444444444444]
[28.68741859 33.25733315] [34.0, 27.33333333333333]
[28.84167741 32.34098622] [33.5, 24.125]
[29.00301189 33.11126142] [34.5, 27.5]

I calculate the coefficient of determination and print the predicted value and real value side by side for predicting mean temperature and humidity, with 'date', 'wind_speed', and 'meanpressure' as input and predicting mean ''wind_speed' and 'meanpressure', with 'date' as input

In [53]:
def my_predict(_x):
    xp = 0
    if isinstance(_x, list):
        xp = pd.DataFrame(_x)
    elif isinstance(_x, pd.DataFrame):
        xp = _x.copy()
    try:
        if xp.shape[1] == 1:
            temp = nnSpeedPressure.predict(xp)
            tempList = []
            for a, b in zip(xp.values.tolist(), temp.tolist()):
                temp = []
                temp.extend(a)
                temp.extend(b)
                tempList.append(temp)
            xp = pd.DataFrame(tempList)
        return nnTempHumid.predict(xp)
    except Exception as e:
        print(str(e))
        return []


If the input for this method has 1 column then it will predict the Wind Speed and Mean Pressure with input as parameter. Then I combine the result of prediction and parameter and use them to predict Temperature and Humidity.

In [54]:
last_day_data = '2017-04-24'
predict_day = '2017-05-01'
# convert string to date object
d1 = datetime.strptime(last_day_data, "%Y-%m-%d")
d2 = datetime.strptime(predict_day, "%Y-%m-%d")

dif = (d2 - d1)
val = (df.shape)[0] + dif.days

print('Predicting temperature and humidity on ', predict_day)
print(my_predict([[val]]))

Predicting temperature and humidity on  2017-05-01
[[30.82233384 27.18330115]]


c:\Users\serey\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
c:\Users\serey\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(


The last date in Climate_dataset_TS.csv is 2017-04-24. I want to know what the mean temperature and humidity value are on 2017-05-01. The encoded value for 2017-04-24 is 114. The difference in day between 2017-04-24 and 2017-05-01 is 7 days. <br/>
so the encoded value for 2017-05-01 is 121. I call the method my_predict() with [[121]] as input and print the value of mean temperature and humidity on that day.

In [55]:
resu = my_predict(X_test[['date']])
score = r2_score(y_test.values.tolist(), resu)
print('Score:', score)
for a, b in zip(resu, y_test.values.tolist()):
    print(a, b)

Score: -0.8686995367122223
[26.2800973  40.30138796] [29.75, 33.75]
[26.4297483  39.81099786] [30.5, 29.75]
[26.5793993  39.32060775] [30.933333333333334, 31.866666666666667]
[26.7290503  38.83021765] [29.23076923076923, 46.0]
[26.8787013  38.33982755] [31.22222222222222, 26.0]
[27.0283523  37.84943744] [27.0, 29.875]
[27.1780033  37.35904734] [25.625, 29.375]
[27.3276543  36.86865724] [27.125, 21.125]
[27.47730531 36.37826713] [27.857142857142858, 19.428571428571427]
[27.62695631 35.88787703] [29.25, 17.75]
[27.77660731 35.39748692] [29.25, 26.0]
[27.92625831 34.90709682] [29.666666666666668, 29.11111111111111]
[28.07590931 34.41670672] [30.5, 37.625]
[28.22736504 33.98088365] [31.22222222222222, 30.444444444444443]
[28.38001026 33.58102586] [31.0, 34.25]
[28.53265548 33.18116807] [32.55555555555556, 38.44444444444444]
[28.68530071 32.78131027] [34.0, 27.33333333333333]
[28.83794593 32.38145248] [33.5, 24.125]
[28.99059115 31.98159468] [34.5, 27.5]
[29.14323638 31.58173689] [34.25, 39

c:\Users\serey\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(


I date values from testing data's to predict for mean temperature and humidity. I want to see how they perform. The method my_predict is will take input with only encoded date value and predict for Wind Speed and Mean Pressure then with the result from the prediction I predict Temperature and Humidity.